In [1]:
import jax
import jax.numpy as jnp
import genjax
genjax.pretty()
import b3d
import b3d.chisight.gen3d as gen3d
import b3d.chisight.gen3d.inference.inference as inference
from tqdm import tqdm

In [2]:
problematic_scene_specs = [(2, 5), (3, 0), (3, 1), (3, 2), (4, 1), (4, 3), (5, 0)]
spec = problematic_scene_specs[2]
scene_id, object_idx = spec

In [ ]:
all_data, meshes, renderer, intrinsics, initial_object_poses = gen3d.dataloading.load_scene(
    scene_id, FRAME_RATE=50, subdir="train_real"
)
b3d.viz_rgb(all_data[0]["rgbd"])

In [4]:
template_pose, model_vertices, model_colors = gen3d.dataloading.load_object_given_scene(
    all_data, meshes, renderer, object_idx
)

In [ ]:
hyperparams = gen3d.settings.hyperparams
hyperparams["intrinsics"] = intrinsics
hyperparams["vertices"] = model_vertices
hyperparams

In [ ]:
inference_hyperparams = gen3d.settings.inference_hyperparams
inference_hyperparams = gen3d.hyperparams.InferenceHyperparams(**{
    **inference_hyperparams.attributes_dict(),
    # "pose_proposal_args": [(0.04, 1000.0)]
})
inference_hyperparams

In [7]:
def get_gt_pose(T):
    return (
        all_data[T]["camera_pose"].inv()
        @ all_data[T]["object_poses"][object_idx]
    )

In [8]:
initial_state = gen3d.dataloading.get_initial_state(
    template_pose, model_vertices, model_colors, hyperparams
)

In [ ]:
key = jax.random.PRNGKey(156)
og_trace = inference.get_initial_trace(
    key, hyperparams, initial_state, all_data[0]["rgbd"]
)

In [10]:
b3d.rr_init("inference_debugging-3-1-2")


In [ ]:
b3d.reload(inference)
b3d.reload(b3d.chisight.gen3d.point_attribute_proposals)

In [ ]:
tracking_results = {}
maxT = 20
key = jax.random.PRNGKey(1)
trace = og_trace
for T in tqdm(range(maxT)):
    key = b3d.split_key(key)
    prev_trace = trace
    trace, _, all_weights, all_poses, keys_to_regen = inference.inference_step(
        key,
        trace,
        all_data[T]["rgbd"],
        inference_hyperparams,
        gt_pose=get_gt_pose(T),
        use_gt_pose=True,
        get_all_weights = True
    )
    tracking_results[T] = trace

    gt_pose, gt_key = all_poses[0], keys_to_regen[0]
    gt_trace = gen3d.inference.get_trace_generated_during_inference(
        gt_key, prev_trace, gt_pose, inference_hyperparams,
        observed_rgbd=all_data[T]["rgbd"]
    )

    gen3d.model.viz_trace(
        trace,
        T,
        ground_truth_vertices=meshes[object_idx].vertices,
        ground_truth_pose=get_gt_pose(T),
    )

    if jnp.linalg.norm(gt_pose.position - gen3d.model.get_new_state(trace)["pose"].position) > 0.02:
        b3d.rr_init("gt_pose_trace-3-1")
        gen3d.model.viz_trace(gt_trace, T, ground_truth_vertices=meshes[object_idx].vertices, ground_truth_pose=get_gt_pose(T))
        break

In [ ]:
trace.get_score()

In [ ]:
gt_trace.get_score()

In [ ]:
gt_trace.subtraces[6]

In [17]:
observed_rgbd = all_data[6]["rgbd"]
state = gen3d.model.get_new_state(gt_trace)
intrinsics = hyperparams["intrinsics"]
(
    observed_rgbd_per_point,
    latent_rgbd_per_point,
    is_valid,
    _,
    point_indices_for_observed_rgbds,
) = gen3d.image_kernel.calculate_latent_and_observed_correspondences(
    observed_rgbd, state, hyperparams
)

In [ ]:
state["depth_nonreturn_prob"][point_indices_for_observed_rgbds[0]]

In [ ]:
scores = jax.vmap(
    hyperparams["image_kernel"].get_rgbd_vertex_kernel().logpdf,
    in_axes=(0, 0, None, None, 0, 0, None, None),
)(
    observed_rgbd_per_point,
    latent_rgbd_per_point,
    state["color_scale"],
    state["depth_scale"],
    state["visibility_prob"][point_indices_for_observed_rgbds],
    state["depth_nonreturn_prob"][point_indices_for_observed_rgbds],
    hyperparams["intrinsics"],
    hyperparams["unexplained_depth_nonreturn_prob"]
)
jnp.isinf(scores)

In [ ]:
observed_rgbd_per_point[-1]

In [ ]:
is_valid[42]

In [ ]:
latent_rgbd_per_point[42]

In [ ]:
observed_rgbd_per_point[42]

In [ ]:
state["visibility_prob"][point_indices_for_observed_rgbds][42]

In [ ]:
state["depth_nonreturn_prob"][point_indices_for_observed_rgbds][42]

In [ ]:
observed_rgbd = observed_rgbd_per_point[0]
latent_rgbd = latent_rgbd_per_point[0]
color_scale = state["color_scale"]
depth_scale = state["depth_scale"]
visibility_prob = state["visibility_prob"][point_indices_for_observed_rgbds][0]
depth_nonreturn_prob = state["depth_nonreturn_prob"][point_indices_for_observed_rgbds][0]
intrinsics = hyperparams["intrinsics"]

hyperparams["image_kernel"].get_rgbd_vertex_kernel().logpdf(
    observed_rgbd,
    latent_rgbd,
    color_scale,
    depth_scale,
    visibility_prob,
    0.,
    intrinsics,
    hyperparams["unexplained_depth_nonreturn_prob"]
)

In [ ]:
_, _, metadata = get_trace_generated_during_inference(
    gt_key, prev_trace, gt_pose, inference_hyperparams,
    observed_rgbd=all_data[T]["rgbd"],
    just_return_trace=False
)
jax.tree.map(lambda x: x[0], metadata["point_attribute_proposal_metadata"])

In [ ]:
metadata["point_attribute_proposal_metadata"]

In [ ]:
hyperparams["image_kernel"].get_rgbd_vertex_kernel().outlier_color_distribution.logpdf(
    observed_rgbd[:3], latent_rgbd[:3], color_scale
)

In [31]:
s = hyperparams["image_kernel"].get_rgbd_vertex_kernel()

In [ ]:
.outlier_depth_distribution.logpdf(
    observed_rgbd[3], latent_rgbd[3], depth_scale, intrinsics["near"], intrinsics["far"]
)

In [86]:
total_log_prob = 0.0
is_depth_non_return = observed_rgbd[3] == 0.0

# Is visible
total_visible_log_prob = 0.0
# color term
total_visible_log_prob += s.inlier_color_distribution.logpdf(
    observed_rgbd[:3], latent_rgbd[:3], color_scale
)
# depth term
total_visible_log_prob += jnp.where(
    is_depth_non_return,
    jnp.log(depth_nonreturn_prob),
    jnp.log(1 - depth_nonreturn_prob)
    + s.inlier_depth_distribution.logpdf(
        observed_rgbd[3],
        latent_rgbd[3],
        depth_scale,
        intrinsics["near"],
        intrinsics["far"],
    ),
)


In [ ]:
jnp.log(1 - depth_nonreturn_prob)

In [ ]:
total_not_visible_log_prob = 0.0
# color term
outlier_color_log_prob = s.outlier_color_distribution.logpdf(
    observed_rgbd[:3],
    latent_rgbd[:3],
    color_scale,
)
outlier_depth_log_prob = s.outlier_depth_distribution.logpdf(
    observed_rgbd[3],
    latent_rgbd[3],
    depth_scale,
    intrinsics["near"],
    intrinsics["far"],
)
outlier_depth_log_prob + outlier_color_log_prob

In [ ]:
total_not_visible_log_prob += outlier_color_log_prob
# depth term
total_not_visible_log_prob += jnp.where(
    is_depth_non_return,
    jnp.log(depth_nonreturn_prob),
    jnp.log(1 - depth_nonreturn_prob) + outlier_depth_log_prob,
)
total_not_visible_log_prob